In [21]:
import pandas as pd
from tqdm import tqdm

In [22]:
frequency = 'monthly'

In [23]:
nyc_master_df = pd.read_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')

In [24]:
nyc_master_df.head()

,Unnamed: 0,year,month,station_complex_id,station_complex,borough,latitude,longitude,ridership,unique_time,...,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,0,2021,6,1,"Astoria-Ditmars Blvd (N,W)",Queens,40.775036,-73.912030,197465.0,2021-06-01,...,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.912034 40.775036)
1,1,2021,6,10,"49 St (N,R,W)",Manhattan,40.759900,-73.984140,197790.0,2021-06-01,...,Subway,40.759901,-73.984139,Uptown,Downtown,2.0,1.0,0.0,Uptown only,POINT (-73.984139 40.759901)
2,2,2021,6,100,"Hewes St (M,J)",Brooklyn,40.706870,-73.953430,46605.0,2021-06-01,...,Elevated,40.706870,-73.953431,Outbound,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.953431 40.70687)
3,3,2021,6,101,"Marcy Av (M,J,Z)",Brooklyn,40.708360,-73.957756,174303.0,2021-06-01,...,Elevated,40.708359,-73.957757,Outbound,Manhattan,1.0,1.0,1.0,NaN,POINT (-73.957757 40.708359)
4,4,2021,6,103,"Bowery (J,Z)",Manhattan,40.720280,-73.993910,46841.0,2021-06-01,...,Subway,40.720280,-73.993915,Brooklyn,Downtown,0.0,0.0,0.0,NaN,POINT (-73.993915 40.72028)


In [25]:
station_network_list = nyc_master_df['GTFS Stop ID'].unique()
station_network_list.sort()
time_interval = nyc_master_df['unique_time'].unique()
time_interval.sort()

In [26]:
def compute_time_series(col: str):

    if col not in nyc_master_df:
        raise ValueError('Column does not exist')
    temp_dict = {}

    for station in tqdm(station_network_list):
        for time in time_interval:
            temp_df = nyc_master_df[(nyc_master_df['GTFS Stop ID'] == station) & (nyc_master_df['unique_time'] == time)]
            val = temp_df[col].mean() # Takes mean of all rows
            if station not in temp_dict:
                temp_dict[station] = {}
            temp_dict[station][time] = val
    return temp_dict

In [27]:
ts_matrix = pd.DataFrame(compute_time_series('ridership'))

100%|██████████| 426/426 [00:00<00:00, 2591.95it/s]


In [28]:
ts_matrix

,101,103,104,106,107,108,109,110,111,113,...,R41,R42,R43,R44,R45,S01,S03,S04,S30,S31
2021-06-01,99679.0,59211.0,129204.0,71692.0,23956.0,88506.0,98851.0,127454.0,18.0,153850.0,...,178589.0,102684.0,71999.0,156892.0,72931.0,72893.0,22106.0,92145.5,9070.0,103735.0


In [29]:
# Shortcut solution to set all NA values to 0
ts_matrix.fillna(0, inplace=True)

# TODO Imputation for NA values

In [30]:
ts_matrix.to_csv(f'Data/Master Sets/NYC Master/NYC_Time_Series_{frequency}.csv')